# F.E.R.B – Fine-tuned Encoder for Response and Behavior

## Step 1: Installing and importing the libraries

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/2

In [2]:
!pip install huggingface_hub

In [3]:
!pip install --upgrade bitsandbytes
!pip install --upgrade transformers accelerate
!pip install --upgrade trl peft datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 11.0 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.40.2
    Uninstalling bitsandbytes-0.40.2:
      Successfully uninstalled bitsandbytes-0.40.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 69.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0
  Attempting uninstall: accelerate
    Found existing installation: accelerat

In [4]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [5]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

## Step 2: Loading the model

In [6]:
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit = True,
                                                                                            bnb_4bit_compute_dtype = getattr(torch, "float16"),
                                                                                            bnb_4bit_quant_type = "nf4"))
llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

## Step 3: Loading the tokenizer

In [7]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "aboonaji/llama2finetune-v2", trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

## Step 4: Setting the training arguments

In [8]:
training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size = 4, max_steps = 100)

In [9]:
torch.cuda.empty_cache()

In [10]:
!nvidia-smi


Wed Feb 12 12:39:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P0             29W /   70W |    4272MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [11]:
!kill -9 $(nvidia-smi | awk '$2=="Processes:" {f=1; next} f {print $5}' | xargs)


/bin/bash: line 1: kill: PID: arguments must be process or job IDs
/bin/bash: line 1: kill: |: arguments must be process or job IDs


In [12]:
!nvidia-smi


Wed Feb 12 12:39:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P0             29W /   70W |    4272MiB /  15360MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:

torch.cuda.empty_cache()


## Step 5: Creating the Supervised Fine-Tuning trainer

In [14]:
from datasets import load_dataset
from peft import LoraConfig
from transformers import TrainingArguments

# ✅ Load and preprocess the dataset
train_dataset = load_dataset("aboonaji/wiki_medical_terms_llam2_format", split="train")

def preprocess_function(examples):
    return llama_tokenizer(
        examples["text"], truncation=True, padding="max_length", max_length=512
    )

# ✅ Apply tokenization and batching
train_dataset = train_dataset.map(preprocess_function, batched=True, num_proc=4)

# ✅ Define LoRA Config (Fixing the missing variable)
peft_config = LoraConfig(
    task_type="CAUSAL_LM", r=64, lora_alpha=16, lora_dropout=0.1
)

# ✅ Define Training Arguments with Fewer Steps
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,  # ✅ Reduce epochs further if needed
    max_steps=30,  # ✅ Stop training after 500 steps (Adjust as needed)
    per_device_train_batch_size=1,  # ✅ Lower batch size to avoid OOM
    gradient_accumulation_steps=8,  # ✅ Simulates larger batch
    fp16=True,  # ✅ Mixed precision training
    save_steps=100,  # ✅ Save model every 100 steps
    save_total_limit=2,
    logging_steps=10,
    report_to="wandb"  # ✅ Logs to Weights & Biases
)

# ✅ Initialize Trainer
llama_sft_trainer = SFTTrainer(
    model=llama_model,
    args=training_arguments,
    train_dataset=train_dataset,
    peft_config=peft_config  # ✅ Now defined correctly
)


wiki_medical_terms_llam2.jsonl:   0%|          | 0.00/54.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6861 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/6861 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(


## Step 6: Training the model

fdd17058c6cc236455c7169f3ebaebfc2628d1aa

In [15]:
llama_sft_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abhigyansdas1234 (abhigyansdas1234-vellore-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
10,5.433700
20,3.886700
30,2.845400


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:716: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 5f1e8690-355e-4937-956d-bf4ebc500023)') - silently ignoring the lookup for the file config.json in NousResearch/Llama-2-7b-chat-hf.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:246: UserWarning: Could not find a config file in NousResearch/Llama-2-7b-chat-hf - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=30, training_loss=4.05527400970459, metrics={'train_runtime': 469.8672, 'train_samples_per_second': 0.511, 'train_steps_per_second': 0.064, 'total_flos': 4896201312829440.0, 'train_loss': 4.05527400970459, 'epoch': 0.034980323567993})

In [16]:
!pip install gtts

In [17]:
!pip install gtts playsound

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=3daac817f4a0ed18c95bb70b95bf836928dde235f63dc4d1e4aa34743edacb87
  Stored in directory: /root/.cache/pip/wheels/50/98/42/62753a9e1fb97579a0ce2f84f7db4c21c09d03bb2091e6cef4
Successfully built playsound


In [21]:
!pip install pydub

## Step 7: Chatting with the model

In [37]:

from gtts import gTTS
import IPython.display as ipd



State = True
user_prompt = input()
while State == True:
  #user_prompt = input()
  if user_prompt.lower()== 'hey ferb':
    print("Type Bye Ferb to exit")
    intro= "F.E.R.B: Hi there! How may I help you?"
    tts = gTTS(text=intro, lang='en')
    tts.save("intro.mp3")
    ipd.display(ipd.Audio("intro.mp3"))
    print(intro)
    query = input()

    if query.lower() == "bye ferb":
      outro="F.E.R.B: Bye!"
      tts = gTTS(text=outro, lang='en')
      tts.save("outro.mp3")
      ipd.display(ipd.Audio("outro.mp3"))
      print(outro)
      State = False
      break

    else:
      text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 300)
      model_answer = text_generation_pipeline(f"<s>[INST] {query} [/INST]")
      print('F.E.R.B: ',model_answer[0]['generated_text'])
      tts = gTTS(text=model_answer[0]['generated_text'], lang='en')
      tts.save("answer.mp3")
      ipd.display(ipd.Audio("answer.mp3"))

  else:
    print("Type Hey Ferb")


hey ferb
Type Bye Ferb to exit


F.E.R.B: Hi there! How may I help you?
symptoms of skin cancer


Device set to use cuda:0


F.E.R.B:  <s>[INST] symptoms of skin cancer [/INST]  Skin cancer can cause a variety of symptoms, including:
 Unterscheidung between different types of skin cancer:

1. Basal cell carcinoma (BCC):
* A small, shiny bump or nodule on the skin that may be pink, red, or the same color as the surrounding skin
* A flat, scaly patch on the skin that may be tan, brown, or black
* A small, painless sore that heals and then comes back
* A red, scaly patch on the skin that may bleed or crust over
* A white, waxy, or pearly bump on the skin
2. Squamous cell carcinoma (SCC):
* A firm, flat, or raised bump on the skin that may be pink, red, or the same color as the surrounding skin
* A scaly patch on the skin that may be tan, brown, or black
* A sore that bleeds or crusts over and doesn't heal
* A rough, scaly, or crusty patch on the skin
* A red, irritated, or itchy patch on the skin
3. Melanoma:
* A new or changing mole that is larger than the size of a pencil eraser (about 6mm


Type Bye Ferb to exit


F.E.R.B: Hi there! How may I help you?
bye ferb


F.E.R.B: Bye!
